### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = ontem

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-11-18



100%|██████████| 299/299 [04:18<00:00,  1.16it/s]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 291 jogos.

----------------------------------------------------------

Atenção! 8 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
130,2023-11-18,Suécia Superettan,13:00,Djurgarden,Hogsbo,NaN,NaN,Não iniciado
291,2023-11-18,Eua Nba,22:00,Milwaukee Bucks,Dallas Mavericks,120.0,117.0,4º Quarto8'
292,2023-11-18,Eua Nba,22:00,San Antonio Spurs,Memphis Grizzlies,99.0,104.0,4º Quarto5'
293,2023-11-18,Eua Ncaa,22:00,California Baptist,St. Thomas (Minn.),NaN,NaN,Não iniciado
294,2023-11-18,Eua Nba,22:00,Chicago Bulls,Miami Heat,84.0,87.0,4º Quarto7'
296,2023-11-18,Eua Ncaa,22:30,Creighton,Texas Southern,NaN,NaN,Não iniciado
297,2023-11-18,Eua Nba,22:30,Golden State Warriors,Oklahoma City Thunder,81.0,65.0,3º Quarto6'
298,2023-11-18,Eua Ncaa,23:00,New Mexico State,Northern Colorado,NaN,NaN,Não iniciado
